In [1]:
module O

"""
抽象平面ベクトルの型
"""
abstract type AbstractPlanarVector{T} end

"""タプル版和集合(重複を削除)"""
tupleunion(x) = Tuple(unique(x))
tupleunion(x, y) = Tuple(unique((x..., y...)))
tupleunion(x, y, z...) = tupleunion(tupleunion(x, y), z...)

"""
抽象平面ベクトルのpublic propertiesは x と y

タブ補完でもx, yしか表示されなくなる。
"""
function Base.propertynames(p::AbstractPlanarVector, private=false)
    public_properties = (:x, :y)
    private && return tupleunion(public_properties, fieldnames(p))
    public_properties
end

"""
抽象平面ベクトルは (x, y) の形式で表示される
"""
Base.show(io::IO, p::AbstractPlanarVector) = show(io, (p.x, p.y))

"""
平面ベクトルの型

x, yをfieldsに持ち、それらがそのままpublic propertiesになる。
"""
struct PlanarVector{T} <: AbstractPlanarVector{T} x::T; y::T end

"""
標準基底の型

x, yをfieldsとして持たず、型のパラメータ i で何番目の基底ベクトルであるかを指定。
"""
struct CanonBasis{T, i} <: AbstractPlanarVector{T} end
function CanonBasis{T}(i) where T
    @assert i == 1 || i == 2
    CanonBasis{T, i}()
end
CanonBasis(i) = CanonBasis{Int}(i)

"""
標準基底のpublic propertiesの取得メソッド

x, yをfieldsとして持たないので定義してやる必要がある。

* i == 1 ⇒ (x, y) = (1, 0)
* i == 2 ⇒ (x, y) = (0, 1)
"""
function Base.getproperty(::CanonBasis{T, i}, name::Symbol) where {T, i}
    name === :x && return ifelse(i == 1, one(T), zero(T))
    name === :y && return ifelse(i == 2, one(T), zero(T))
    error("type CanonBasis has no public property $name")
end

########## ベクトルの演算達

for op in (:+, :-)
    @eval Base.$op(p::AbstractPlanarVector) = PlanarVector($op(p.x), $op(p.y))
    @eval Base.$op(p::AbstractPlanarVector{T}, q::AbstractPlanarVector{U}) where {T, U} =
        PlanarVector{promote_type(T, U)}($op(p.x, q.x), $op(p.y, q.y))
end

Base.:*(a::T, p::AbstractPlanarVector{U}) where {T, U} =
    PlanarVector{promote_type(T, U)}(*(a, p.x), *(a, p.y))
Base.:*(p::AbstractPlanarVector{U}, a::T) where {T, U} =
    PlanarVector{promote_type(T, U)}(*(p.x, a), *(p.y, a))
Base.:\(a, p::AbstractPlanarVector) = inv(a) * p
Base.:/(p::AbstractPlanarVector, a) = p * inv(a)

using LinearAlgebra

LinearAlgebra.dot(p::AbstractPlanarVector, q::AbstractPlanarVector) =
    conj(p.x) * q.x + conj(p.y) * q.y
LinearAlgebra.dot(p::CanonBasis{T, i}, q::AbstractPlanarVector) where {T, i} =
    i == 1 ? q.x : q.y
LinearAlgebra.dot(p::AbstractPlanarVector, q::CanonBasis{T, i}) where {T, i} =
    conj(i == 1 ? p.x : p.y)
LinearAlgebra.dot(p::CanonBasis{T, i}, q::CanonBasis{U, j}) where {T, i, U, j} =
    (P = promote_type(T, U); i == j ? one(P) : zero(P))

end

Main.O

In [2]:
p = O.PlanarVector(2, 3)

(2, 3)

In [3]:
q = O.PlanarVector(-5.0, 10.0)

(-5.0, 10.0)

In [4]:
p + q

(-3.0, 13.0)

In [5]:
-3p + 4q

(-26.0, 31.0)

In [6]:
O.dot(p, q)

20.0

In [7]:
e1 = O.CanonBasis(1)

(1, 0)

In [8]:
e2 = O.CanonBasis(2)

(0, 1)

In [9]:
O.dot(e1, e2)

0

In [10]:
O.dot(O.CanonBasis{ComplexF64}(1), e1)

1.0 + 0.0im

In [11]:
e1 - e2

(1, -1)

In [12]:
3*e1 - e2/2

(3.0, -0.5)

In [13]:
p.x, p.y

(2, 3)

In [14]:
e1.x, e1.y

(1, 0)

In [15]:
getx(p::O.AbstractPlanarVector) = p.x
gety(p::O.AbstractPlanarVector) = p.y

gety (generic function with 1 method)

In [16]:
getx(p), gety(p)

(2, 3)

In [17]:
@code_typed getx(p)

CodeInfo(
1 ─ %1 = Base.getfield(p, :x)::Int64
└──      return %1
) => Int64

In [18]:
@code_typed gety(q)

CodeInfo(
1 ─ %1 = Base.getfield(p, :y)::Float64
└──      return %1
) => Float64

In [19]:
@code_typed getx(e1)

CodeInfo(
1 ─     return 1
) => Int64

In [20]:
@code_typed gety(e1)

CodeInfo(
1 ─     return 0
) => Int64

In [21]:
@code_typed O.dot(p, q)

CodeInfo(
1 ─ %1 = Base.getfield(p, :x)::Int64
│   %2 = Base.getfield(q, :x)::Float64
│   %3 = Base.sitofp(Float64, %1)::Float64
│   %4 = Base.mul_float(%3, %2)::Float64
│   %5 = Base.getfield(p, :y)::Int64
│   %6 = Base.getfield(q, :y)::Float64
│   %7 = Base.sitofp(Float64, %5)::Float64
│   %8 = Base.mul_float(%7, %6)::Float64
│   %9 = Base.add_float(%4, %8)::Float64
└──      return %9
) => Float64

In [22]:
@code_typed O.dot(e1, q)

CodeInfo(
1 ─ %1 = Base.getfield(q, :x)::Float64
└──      return %1
) => Float64

In [23]:
@code_typed O.dot(p, e2)

CodeInfo(
1 ─      goto #3 if not false
2 ─      nothing::Nothing
3 ┄ %3 = Base.getfield(p, :y)::Int64
└──      return %3
) => Int64

In [24]:
@code_llvm O.dot(p, e2)

;  @ In[1]:83 within `dot`
; Function Attrs: uwtable
define i64 @julia_dot_1882([2 x i64]* nocapture nonnull readonly align 8 dereferenceable(16) %0) #0 {
top:
; ┌ @ Base.jl:42 within `getproperty`
   %1 = getelementptr inbounds [2 x i64], [2 x i64]* %0, i64 0, i64 1
; └
  %2 = load i64, i64* %1, align 8
  ret i64 %2
}


In [25]:
@code_typed O.dot(e1, e1)

CodeInfo(
1 ─     return 1
) => Int64

In [26]:
@code_typed O.dot(e1, e2)

CodeInfo(
1 ─     goto #3 if not false
2 ─     nothing::Nothing
3 ┄     return 0
) => Int64

In [27]:
@code_llvm O.dot(e1, e2)

;  @ In[1]:85 within `dot`
; Function Attrs: uwtable
define i64 @julia_dot_1912() #0 {
top:
  ret i64 0
}


In [28]:
using BenchmarkTools
n = 10^6
v = [O.PlanarVector(randn(2)...) for _ in 1:n]
a = O.PlanarVector(1, 0)
e = O.CanonBasis(1)

@show sum(Base.Fix1(O.dot, a), v) == sum(Base.Fix1(O.dot, e), v)
@btime sum($(Base.Fix1(O.dot, a)), $v)
@btime sum($(Base.Fix1(O.dot, e)), $v)

sum(Base.Fix1(O.dot, a), v) == sum(Base.Fix1(O.dot, e), v) = true
  696.300 μs (0 allocations: 0 bytes)
  643.000 μs (0 allocations: 0 bytes)


146.50529042279067

In [29]:
@code_native debuginfo=:none O.dot(a, q)

	.text
	pushq	%rbp
	movq	%rsp, %rbp
	vcvtsi2sdq	(%rcx), %xmm0, %xmm0
	vmulsd	(%rdx), %xmm0, %xmm0
	vcvtsi2sdq	8(%rcx), %xmm1, %xmm1
	vmulsd	8(%rdx), %xmm1, %xmm1
	vaddsd	%xmm1, %xmm0, %xmm0
	popq	%rbp
	retq
	nop


In [30]:
@code_native debuginfo=:none O.dot(e, q)

	.text
	pushq	%rbp
	movq	%rsp, %rbp
	vmovsd	(%rcx), %xmm0                   # xmm0 = mem[0],zero
	popq	%rbp
	retq
	nopw	(%rax,%rax)
